In [17]:
import numpy as np
import nibabel as nib
from utils.analysis import compute_dice

seg = nib.load('/home/t722s/Desktop/ExperimentResults/sammed3d_abdomenAtlas_20240723_1546/random_points/kidney_left/BDMAP_00000001_seg.nii.gz')
seg = seg.get_fdata().astype(np.uint8)
gt_nib = nib.load('/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz')
gt = gt_nib.get_fdata().astype(np.uint8)
gt = np.where(gt==3, 1, 0)

In [18]:
compute_dice(gt, seg)

0.8505915492957746